In [7]:
from kan.hypothesis import *
from torchvision import transforms
import torch
import torch.nn.functional as F
from kan import *
import h5py
import copy
import sympy as sp
dtype = torch.get_default_dtype()
from sympy import latex
import pickle

In [8]:
data_dir = "./"
filename = "sg2-data.h5"
checkpoint_dir = "./saved_models/"
log_dir = './logs/'
band_idx = 0
gidxs = [1, 2, 3, 4]
only_obstructed = False
only_topological = False
binary_classification = False#True
no_penalize_last = True
bias = False
numrs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
file = h5py.File(data_dir + filename, 'r')
print("file keys: ", file['sg2/1/'].keys());

file keys:  <KeysViewHDF5 ['epsilon_Gs-gidx=1', 'epsilon_Gs-gidx=2', 'epsilon_Gs-gidx=3', 'epsilon_Gs-gidx=4', 'frequencies-gidx=1-mode=tm', 'frequencies-gidx=2-mode=tm', 'frequencies-gidx=3-mode=tm', 'frequencies-gidx=4-mode=tm', 'symmetry-gidx=1-mode=tm', 'symmetry-gidx=2-mode=tm', 'symmetry-gidx=3-mode=tm', 'symmetry-gidx=4-mode=tm']>


In [5]:
nGs = 10
gidxs = [1, 2, 3, 4]
wps = ['1a', '1b', '1c', '1d']
wps2 = ['1a', '1c', '1b', '1d'] # switch 1b and 1c due to convention of how Gvectors are stored

symmetry_data = torch.zeros(10000 * len(gidxs) * len(wps))
input_data = torch.zeros(10000 * len(gidxs) * len(wps), nGs)

In [6]:
for (widx, (wp1, wp2)) in enumerate(zip(wps, wps2)):
    print(f"Wyckoff index: {widx}")
    sym_vec_phases = file[f'sg2/symmetry_vector_phases/{wp1}'][()]
    epsilon_G_phases = file[f'sg2/epsilon_G_phases/{wp2}'][()].real
    for gidx in gidxs:
        print(f"gidx: {gidx}")
        for id in range(1, 10001):
            real_id = (id-1) + (gidx-1)*10000 + len(gidxs)*10000*widx
            if (id % 1000 == 0): 
                print(id)
            symmetry_before_aug = file[f'sg2/{id}/symmetry-gidx={gidx}-mode=tm'][()][band_idx]
            if band_idx == 0:
                symmetry_data[real_id] =  sym_vec_phases[symmetry_before_aug] - 8
            else: 
                symmetry_data[real_id] =  sym_vec_phases[symmetry_before_aug] 

            fourier_data_before_aug = file[f'sg2/{id}/epsilon_Gs-gidx={gidx}'][()][0:nGs].real
            fourier_data = fourier_data_before_aug * epsilon_G_phases
            input_data[real_id, :] = torch.tensor([*fourier_data])

Wyckoff index: 0
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 1
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 2
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 3
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000

In [10]:
shuffled_indices = torch.randperm(10000 * len(gidxs) * len(wps))
input_data_shuffled = input_data[shuffled_indices, :]
symmetry_data_shuffled = symmetry_data[shuffled_indices]

In [11]:
dataset_kan = {}
n_train = 80000#int(input_data_shuffled.shape[0]*2/3)
nGs = 10
nGstart = 1
dataset_kan['train_input'] = input_data_shuffled[0:n_train, [*range(nGstart, nGs)]].to(device)
dataset_kan['test_input'] = input_data_shuffled[n_train:, [*range(nGstart, nGs)]].to(device)
dataset_kan['train_label'] = symmetry_data_shuffled[0:n_train].long().to(device)
dataset_kan['test_label'] = symmetry_data_shuffled[n_train:].long().to(device)

In [12]:
all_formulas = []
for topological_class in range(0, 8): 
    with open(f"formula-class{topological_class}.pkl", "rb") as f:
        loaded_formula = pickle.load(f)
        all_formulas.append(loaded_formula)

In [13]:
all_formulas_rounded = [ex_round(x, 3) for x in all_formulas]

In [14]:
all_formulas_rounded[0]

-129.609*sin(37.858*(-0.009*x_1 - 1)**2 + 0.269*(-0.14*x_2 - 1)**2 - 45.538 + 0.03*exp(-1.591*(1 - 0.379*x_3)**2)) + 2969.801*cos(0.773*x_2 - 0.04*(1 - 0.049*x_3)**2 + 0.005*(-0.953*x_1 - 1)**2 - 9.719) - 2658.012*cos(0.011*(1 - 0.035*x_2)**2 + 0.001*sin(0.909*x_1 + 7.796) - 11.023*sin(0.055*x_3 + 9.374) + 10.277) - 2359.975*cos(8.398*sin(0.027*x_1 + 9.295) + 0.005*tanh(0.482*x_3 - 0.682) - 10.845 + 0.003*exp(-1.608*(1 - 0.393*x_2)**2)) - 2084.424

In [15]:
for i in range(0, 8):
    print(latex(all_formulas_rounded[i]), "\n\n")

- 129.609 \sin{\left(37.858 \left(- 0.009 x_{1} - 1\right)^{2} + 0.269 \left(- 0.14 x_{2} - 1\right)^{2} - 45.538 + 0.03 e^{- 1.591 \left(1 - 0.379 x_{3}\right)^{2}} \right)} + 2969.801 \cos{\left(0.773 x_{2} - 0.04 \left(1 - 0.049 x_{3}\right)^{2} + 0.005 \left(- 0.953 x_{1} - 1\right)^{2} - 9.719 \right)} - 2658.012 \cos{\left(0.011 \left(1 - 0.035 x_{2}\right)^{2} + 0.001 \sin{\left(0.909 x_{1} + 7.796 \right)} - 11.023 \sin{\left(0.055 x_{3} + 9.374 \right)} + 10.277 \right)} - 2359.975 \cos{\left(8.398 \sin{\left(0.027 x_{1} + 9.295 \right)} + 0.005 \tanh{\left(0.482 x_{3} - 0.682 \right)} - 10.845 + 0.003 e^{- 1.608 \left(1 - 0.393 x_{2}\right)^{2}} \right)} - 2084.424 


54.055 \sin{\left(65.227 \left(- 0.009 x_{1} - 1\right)^{2} + 0.464 \left(- 0.14 x_{2} - 1\right)^{2} - 73.133 + 0.051 e^{- 1.591 \left(1 - 0.379 x_{3}\right)^{2}} \right)} - 2813.432 \cos{\left(0.385 x_{2} - 0.02 \left(1 - 0.049 x_{3}\right)^{2} + 0.003 \left(- 0.953 x_{1} - 1\right)^{2} - 9.767 \right)} - 2273

In [16]:
def formula_eval(t, formula_idx):
    x, y, z = t
    return float(all_formulas[formula_idx].subs('x_1', x).subs('x_2', y).subs('x_3', z))
def formula_eval_rounded(t, formula_idx):
    x, y, z = t
    return float(all_formulas_rounded[formula_idx].subs('x_1', x).subs('x_2', y).subs('x_3', z))

In [17]:
N = 100 #symmetry_data.shape[0]
formula_predictions = torch.zeros((N, 8), dtype=float)
for i in range(0, N): 
    if ((i+1) % 100) == 0:
        print(f"i={i+1}", flush=true)
    for j in range(0, 8):
        y = formula_eval_rounded(input_data[i:i+1, 1:4][0], j)
        formula_predictions[i, j] = y

i=100


In [18]:
(formula_predictions.argmax(1) == symmetry_data[0:N]).sum()/N

tensor(0.9900)